In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import sys


project_root = Path.cwd()
while not (project_root / "src").exists() and project_root != project_root.parent:
    project_root = project_root.parent

src_path = project_root / "src"
fig_dir = project_root / "figures"
res_dir = project_root / "results"

fig_dir.mkdir(exist_ok=True)
res_dir.mkdir(exist_ok=True)

print("CWD:", Path.cwd())
print("project_root:", project_root)
print("src_path:", src_path, "exists:", src_path.exists())

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from model_proteostasis import (
    ModelParams,
    simulate_population,
    survival_curve,
    collapse_stats,        
)


CWD: C:\Users\Regi\Documents\Replication-Asymmetry-Model\notebooks
project_root: C:\Users\Regi\Documents\Replication-Asymmetry-Model
src_path: C:\Users\Regi\Documents\Replication-Asymmetry-Model\src exists: True


In [3]:
p = ModelParams()

d_val     = 0.8
gamma_val = 0.0012
B_val     = 2000.0
Lcrit_val = 2e4
T_max     = 2000
reps_val  = 3000

coll0, t0, coll1, t1 = simulate_population(
    d=d_val,
    gamma=gamma_val,
    B=B_val,
    Lcrit=Lcrit_val,
    T=T_max,
    reps=reps_val,
    p=p,
)

stats_H0 = collapse_stats(t0, "H0")
stats_H1 = collapse_stats(t1, "H1")

frac_coll_H0 = np.mean(coll0)
frac_coll_H1 = np.mean(coll1)

stats_H0, stats_H1, frac_coll_H0, frac_coll_H1


({'label': 'H0', 'n': 612, 'median': 1249.0, 'q25': 914.75, 'q75': 1593.5},
 {'label': 'H1', 'n': 2952, 'median': 833.0, 'q25': 697.75, 'q75': 1059.25},
 np.float64(0.204),
 np.float64(0.984))

In [4]:
row = {
    "d": d_val,
    "gamma": gamma_val,
    "B": B_val,
    "Lcrit": Lcrit_val,
    "T": T_max,
    "reps": reps_val,
    "H0_n": stats_H0["n"],
    "H0_median": stats_H0["median"],
    "H0_q25": stats_H0["q25"],
    "H0_q75": stats_H0["q75"],
    "H0_frac_collapsed": float(frac_coll_H0),
    "H1_n": stats_H1["n"],
    "H1_median": stats_H1["median"],
    "H1_q25": stats_H1["q25"],
    "H1_q75": stats_H1["q75"],
    "H1_frac_collapsed": float(frac_coll_H1),
}

df_stress = pd.DataFrame([row])
out_path = res_dir / "population_collapse_stress_test.csv"
df_stress.to_csv(out_path, index=False)
print("Saved:", out_path)


Saved: C:\Users\Regi\Documents\Replication-Asymmetry-Model\results\population_collapse_stress_test.csv


In [5]:
S0 = survival_curve(t0, T=T_max)
S1 = survival_curve(t1, T=T_max)

plt.figure()
plt.plot(np.arange(1, T_max+1), S0, label="H0 (symmetric)")
plt.plot(np.arange(1, T_max+1), S1, label="H1 (asymmetric)")
plt.xlabel("Time (divisions)")
plt.ylabel("Fraction surviving")
plt.title(f"Stress-test survival (d={d_val}, gamma={gamma_val}, Lcrit={Lcrit_val:.1e})")
plt.legend()

fig_path = fig_dir / "survival_stress_test.png"
plt.savefig(fig_path, dpi=300, bbox_inches="tight")
plt.close()
print("Saved:", fig_path)


Saved: C:\Users\Regi\Documents\Replication-Asymmetry-Model\figures\survival_stress_test.png
